参考文档：
https://colab.research.google.com/github/openai/openai-cookbook/blob/master/examples/vector_databases/pinecone/GPT4_Retrieval_Augmentation.ipynb

In [2]:
!pip install -qU bs4 tiktoken openai langchain pinecone-client[grpc]


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!wget -r -A.html -P rtdocs http://docs.sourcefabric.org/projects/ebooklib/en/latest/

--2023-05-04 08:28:47--  http://docs.sourcefabric.org/projects/ebooklib/en/latest/
Resolving docs.sourcefabric.org (docs.sourcefabric.org)... 104.17.33.82, 104.17.32.82, 2606:4700::6811:2052, ...
Connecting to docs.sourcefabric.org (docs.sourcefabric.org)|104.17.33.82|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘rtdocs/docs.sourcefabric.org/projects/ebooklib/en/latest/index.html’

docs.sourcefabric.o     [ <=>                ]  10.53K  --.-KB/s    in 0s      

2023-05-04 08:28:47 (233 MB/s) - ‘rtdocs/docs.sourcefabric.org/projects/ebooklib/en/latest/index.html’ saved [10783]

Loading robots.txt; please ignore errors.
--2023-05-04 08:28:47--  http://docs.sourcefabric.org/robots.txt
Reusing existing connection to docs.sourcefabric.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 96 [text/plain]
Saving to: ‘rtdocs/docs.sourcefabric.org/robots.txt.tmp’

docs.sourcefabric.o 100%[===================>]      96  --.-

In [4]:
from langchain.document_loaders import ReadTheDocsLoader

loader = ReadTheDocsLoader('rtdocs')
docs = loader.load()
len(docs)

/home/codespace/.python/current/lib/python3.10/site-packages/langchain/document_loaders/readthedocs.py:30: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 30 of the file /home/codespace/.python/current/lib/python3.10/site-packages/langchain/document_loaders/readthedocs.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  _ = BeautifulSoup(
/home/codespace/.python/current/lib/python3.10/site-packages/langchain/document_loaders/readthedocs.py:46: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on anoth

28

In [5]:
docs[0]

Document(page_content='', metadata={'source': 'rtdocs/docs.sourcefabric.org/robots.txt.tmp'})

In [8]:
print(docs[11].page_content)

Plugins¶
Plugins can be used to automate some of the workflow. For instance ebooklib.plugins.tidyhtml plugins automatically cleans HTML
output for you. FootnotePlugin for instance rewrites custom footnotes into EPUB3 kind of footnotes. Without these plugins you would
need to do wanted transformations before setting or getting content from your chapters.
There is a ebooklib.plugins.base.BasePlugin class which you need to extend. These are the methods you can override.
Method
Arguments
Description
before_write
book
Processing before book save
after_write
book
Processing after book save
before_read
book
Processing before book read
after_read
book
Processing after book read
item_after_read
book, item
Process general item after read
item_before_write
book, item
Process general item before write
html_after_read
book, chapter
Processing HTML before read
html_before_write
book, chapter
Processing HTML before save
Custom plugin¶
This is our use case. We have online WYSIWYG editing system for ed

In [20]:

print("1",docs[5].metadata['source'])
new = docs[5].metadata['source'].replace('rtdocs/', 'https://')


print("2",new)

1 rtdocs/docs.sourcefabric.org/projects/live-blog-restful-api/en/latest/python-example.html
2 https://docs.sourcefabric.org/projects/live-blog-restful-api/en/latest/python-example.html


In [9]:
import tiktoken

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [23]:
chunks = []

for idx, record in enumerate(tqdm(docs)):
    texts = text_splitter.split_text(record.page_content)
    chunks.extend([{
        'id': str(uuid4()),
        'text': texts[i],
        'chunk': i,
        'url': record.metadata['source']  # 替换此行
    } for i in range(len(texts))])


  0%|          | 0/28 [00:00<?, ?it/s]

In [36]:

import openai

# initialize openai API key
openai.api_key = input("sk-...")  #platform.openai.com

embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=embed_model
)

In [25]:
res.keys()

dict_keys(['object', 'data', 'model', 'usage'])

In [26]:
len(res['data'])

2

In [27]:

len(res['data'][0]['embedding']), len(res['data'][1]['embedding'])

(1536, 1536)

In [44]:
import pinecone
import os

index_name = 'gpt-3.5-turo-langchain-docs'

# initialize connection to pinecone
pinecone.init(
    api_key=input("PINECONE_API_KEY"),
    environment=input("PINECONE_ENVIRONMENT")
)


In [46]:
print("出错?",pinecone)
# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='dotproduct'
    )
# connect to index
index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()

出错? <module 'pinecone' from '/home/codespace/.python/current/lib/python3.10/site-packages/pinecone/__init__.py'>


MaxRetryError: HTTPSConnectionPool(host='controller.wangshiqi.pinecone.io', port=443): Max retries exceeded with url: /databases (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f150d73ffd0>: Failed to establish a new connection: [Errno -2] Name or service not known'))